In [13]:
import requests
import random
from collections import Counter
import math

In [15]:
class JogoDeForca:
    def __init__(self):
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code == 200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro: ", r.status_code)

    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content).lower()
        return len(self.palavra)

    def tentar_letra(self, letra):
        letra = letra.lower()
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx] == letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    return False
                else:
                    return []
        else:
            return False

    def tentar_palavra(self, palavra):
        palavra = palavra.lower()
        if self.vidas > 0:
            if self.palavra == palavra:
                return True
            else:
                self.vidas = 0
                return False
        else:
            return False

In [16]:
url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
response = requests.get(url)
if response.status_code == 200:
    vocabulario = response.content.decode('utf-8').splitlines()
    vocabulario = [palavra.lower() for palavra in vocabulario if palavra.isalpha()]
else:
    print("Erro ao carregar o vocabulário.")

In [17]:
def calcular_entropia(letras_possiveis, palavras_possiveis):
    total_palavras = len(palavras_possiveis)
    frequencias = Counter()
    for palavra in palavras_possiveis:
        letras_unicas = set(palavra)
        for letra in letras_unicas:
            if letra in letras_possiveis:
                frequencias[letra] += 1
    entropias = {}
    for letra, freq in frequencias.items():
        p = freq / total_palavras
        entropias[letra] = -p * math.log2(p)
    return entropias

In [18]:
def filtrar_palavras(palavras_possiveis, posicoes_letras, letras_erradas):
    palavras_filtradas = []
    for palavra in palavras_possiveis:
        match = True
        for idx, letra in enumerate(palavra):
            if posicoes_letras[idx]:
                if letra != posicoes_letras[idx]:
                    match = False
                    break
            else:
                if letra in letras_erradas:
                    match = False
                    break
        if match:
            palavras_filtradas.append(palavra)
    return palavras_filtradas

In [19]:
def jogador_automatico(jogo, vocabulario):
    tamanho_palavra = jogo.novo_jogo()
    palavras_possiveis = [palavra for palavra in vocabulario if len(palavra) == tamanho_palavra]
    
    posicoes_letras = [None] * tamanho_palavra
    letras_erradas = set()
    letras_tentadas = set()
    
    while jogo.vidas > 0:
        letras_possiveis = set('abcdefghijklmnopqrstuvwxyz') - letras_tentadas
        entropias = calcular_entropia(letras_possiveis, palavras_possiveis)
        if not entropias:
            palavra_tentativa = palavras_possiveis[0]
            resultado = jogo.tentar_palavra(palavra_tentativa)
            return resultado
        proxima_letra = max(entropias, key=entropias.get)
        letras_tentadas.add(proxima_letra)
        resultado = jogo.tentar_letra(proxima_letra)
        if resultado == False:
            return False
        elif resultado == True:
            return True
        elif resultado:
            for idx in resultado:
                posicoes_letras[idx] = proxima_letra
        else:
            letras_erradas.add(proxima_letra)
        palavras_possiveis = filtrar_palavras(palavras_possiveis, posicoes_letras, letras_erradas)
        if len(palavras_possiveis) == 1:
            palavra_tentativa = palavras_possiveis[0]
            resultado = jogo.tentar_palavra(palavra_tentativa)
            return resultado
    return False

In [20]:
def executar_simulacoes(n_simulacoes, vocabulario):
    vitorias = 0
    detalhes = []
    for _ in range(n_simulacoes):
        jogo = JogoDeForca()
        jogo.novo_jogo()
        resultado = jogador_automatico(jogo, vocabulario)
        detalhes.append({
            'palavra': jogo.palavra,
            'resultado': 'Vitória' if resultado else 'Derrota',
            'vidas_restantes': jogo.vidas
        })
        if resultado:
            vitorias += 1
    probabilidade_vitoria = vitorias / n_simulacoes
    print(f"Total de vitórias: {vitorias} em {n_simulacoes} jogos.")
    print(f"Probabilidade de vitória: {probabilidade_vitoria * 100:.2f}%")
    return probabilidade_vitoria, detalhes

In [ ]:
n_simulacoes = 100

probabilidade_vitoria, detalhes = executar_simulacoes(n_simulacoes, vocabulario)

Total de vitórias: 43 em 100 jogos.
Probabilidade de vitória: 43.00%


In [22]:
derrotas = [detalhe for detalhe in detalhes if detalhe['resultado'] == 'Derrota']

print(f"\nNúmero de derrotas: {len(derrotas)}\n")

for idx, derrota in enumerate(derrotas, 1):
    print(f"{idx}. Palavra: {derrota['palavra']}, Vidas Restantes: {derrota['vidas_restantes']}")


Número de derrotas: 57

1. Palavra: saponaceo, Vidas Restantes: 0
2. Palavra: influencies, Vidas Restantes: 0
3. Palavra: exibirem, Vidas Restantes: 0
4. Palavra: lousa, Vidas Restantes: 0
5. Palavra: reabrira, Vidas Restantes: 0
6. Palavra: emplacaram, Vidas Restantes: 0
7. Palavra: refinares, Vidas Restantes: 0
8. Palavra: vassouremos, Vidas Restantes: 0
9. Palavra: emudecera, Vidas Restantes: 0
10. Palavra: bracejava, Vidas Restantes: 0
11. Palavra: absolvires, Vidas Restantes: 0
12. Palavra: atordoaveis, Vidas Restantes: 0
13. Palavra: praguejaram, Vidas Restantes: 0
14. Palavra: cingirieis, Vidas Restantes: 0
15. Palavra: maximizaveis, Vidas Restantes: 0
16. Palavra: sossobraramos, Vidas Restantes: 0
17. Palavra: referencies, Vidas Restantes: 0
18. Palavra: emocionarei, Vidas Restantes: 0
19. Palavra: sofismares, Vidas Restantes: 0
20. Palavra: efetuarieis, Vidas Restantes: 0
21. Palavra: contornarmos, Vidas Restantes: 0
22. Palavra: apinharieis, Vidas Restantes: 0
23. Palavra: d